In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 636 kB 48.1 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from torch import nn
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math
import time


In [3]:
torch.cuda.is_available()

False

In [4]:
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

In [5]:
class BertEmbedding:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
        # Put the model in "evaluation" mode, meaning feed-forward operation.
        self.model.eval()

    def getEmbeddings(self,text):
      tokens_tensor,segments_tensor = self.TextPreprocessing(text)
      embeddings = self.generateEmbedding(tokens_tensor,segments_tensor)
      return embeddings

    def TextPreprocessing(self,text):
        marked_text = "[CLS] " + text + " [SEP]"
        # Tokenize our sentence with the BERT tokenizer.
        tokenized_text = self.tokenizer.tokenize(marked_text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        return tokens_tensor,segments_tensors
    
    def generateEmbedding(self,tokens_tensor,segments_tensor):
        with torch.no_grad():
            outputs = self.model(tokens_tensor, segments_tensor)
            hidden_states = outputs[2]
        tokens = torch.stack(hidden_states,dim=0)
        tokens = tokens.permute(1,2,0,3)
        final_val = tokens[:,1:-1,-2,:]
        return final_val

In [6]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [7]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [8]:
class MultiHeadedAttention(nn.Module):
    "Multi-headed Attention module"
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [9]:
class FeedForwardLayer(nn.Module):
    "Feedforward with 1 hidden layer"
    def __init__(self,inp_dim,hid_dim,dropout = 0.1):
        super(FeedForwardLayer, self).__init__()
        self.inp_dim = inp_dim
        self.hid_dim = hid_dim
        self.hidden = nn.Linear(inp_dim,hid_dim)
        self.output = nn.Linear(hid_dim,inp_dim)
        self.relu = F.relu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

In [10]:
class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [11]:
class SublayerConnection(nn.Module):
    "Apply residual connection to any sublayer with the same size."
    def __init__(self, size, dropout=0.1):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        
        return x + self.dropout(sublayer(self.norm(x)))
        # return x + sublayer(self.norm(x))


In [12]:
class EncoderBlock(nn.Module):
    "An Encoder block that find the connection between Answer and Question"
    def __init__(self,attentionBlock,feedForwardBlock,size,dropout = 0.1):
        super(EncoderBlock,self).__init__()
        self.attentionBlock = attentionBlock
        self.feedForwardBlock = feedForwardBlock
        self.sublayer = clones(SublayerConnection(size,dropout),2)
        self.size = size

    def forward(self,Query,Value):
        x = self.sublayer[0](Query, lambda x: self.attentionBlock(query=x, value=Value, key=Value))
        return self.sublayer[1](x, self.feedForwardBlock)


In [13]:
class EncoderModule(nn.Module):
    "Stacks of Encoder blocks"
    def __init__(self, EncoderLayer,N):
        super(EncoderModule,self).__init__()
        self.layers = clones(EncoderLayer,N)
        self.norm = LayerNorm(EncoderLayer.size)
    
    def forward(self,Query,Value):
        for layers in self.layers:
            Query = layers(Query = Query,Value = Value)
        return Query


In [14]:
class RepresentationModule(nn.Module):
    "Bottom layer that gives 2 representations: Reference answer Rep and Student answer Rep"
    def __init__(self,EncoderModule, embeddingLayer):
        super(RepresentationModule,self).__init__()
        self.EncoderModules = clones(EncoderModule,2)
        self.embeddingLayer = embeddingLayer
        #Get 2 clones of EncoderModule
    def forward(self, Question, ReferenceAnswer, StudentAnswer):
        Q = self.embeddingLayer.getEmbeddings(Question)
        seconds = time.time()
        StuAns = self.embeddingLayer.getEmbeddings(StudentAnswer)
        RefAns = self.embeddingLayer.getEmbeddings(ReferenceAnswer)
        studentAnsRep = self.EncoderModules[0](Q,StuAns)
        RefAnsRep = self.EncoderModules[1](Q,RefAns)
        return (studentAnsRep,RefAnsRep)

In [15]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [16]:
class FinalFeedForwardLayer(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size,100)
        self.fc2 = nn.Linear(100,1)
    def forward(self, xb):
        xb = F.relu(self.fc1(xb))
        xb = torch.sigmoid(self.fc2(xb))      # batch wise forwarding
        return xb
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)         
        # Calcuate loss
        loss = F.l1_loss(out, targets)  # batch wise training step and loss
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss =F.l1_loss(out, targets)       # batch wise validation and loss    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine val losses of all batches as average
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 100 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [17]:
class PrepLayer(nn.Module):
    def __init__(self,d_model,max_size,positionalLayer):
        super(PrepLayer,self).__init__()
        self.borderLayer = torch.zeros(1,1,768)
        self.d_model = d_model
        self.max_size = max_size
        self.positionalEncoding = positionalLayer
    def forward(self,StudAns,RefAns):
        RefAns = torch.add(RefAns,1)
        FinalRep = torch.cat((StudAns,self.borderLayer),dim=1)
        FinalRep = torch.cat((FinalRep,RefAns),dim = 1)
        FinalRep = self.positionalEncoding(FinalRep)
        FinalRep = FinalRep.flatten(start_dim=1, end_dim=2)
        self.padValue = torch.zeros(1,self.d_model*self.max_size*2+1 - FinalRep.shape[1])
        FinalRep = torch.cat((FinalRep,self.padValue),dim = 1)
        return FinalRep

In [18]:
class GetMarks(nn.Module):
    def __init__(self):
        super(GetMarks,self).__init__()
    def forward(self,grade,full_marks):
        return grade*full_marks

In [19]:
class GradingModule(nn.Module):
    "Uses the Representations to compare and grade them"
    def __init__(self,d_model,max_size,prepLayer,feedForward):
        super(GradingModule,self).__init__()
        self.max_size = max_size
        self.d_model = d_model
        self.prepLayer = prepLayer
        self.feedForward = feedForward
        self.GetMarks = GetMarks()
        
    def forward(self,stu,ref,full_marks):
        FinalRep = self.prepLayer(stu,ref)
        grade = self.feedForward(FinalRep)
        print(grade.size())
        final_marks = self.GetMarks(grade,full_marks)
        return final_marks
        

In [20]:
class Upgrader(nn.Module):
    def __init__(self,Representation_Module, Grading_Module):
        super(Upgrader,self).__init__()
        self.RepModule = Representation_Module
        self.GradModule = Grading_Module
    def forward(self, StudentAns,Question,RefAnswer,full_marks):
        stu_rep,ref_rep = self.RepModule( StudentAns,Question,RefAnswer)
        grad = self.GradModule(stu_rep,ref_rep,full_marks)
        return grad

In [21]:
def makeModel(emb_dim = 768,heads = 3,hid_lay_dim = 768, N = 6,max_size = 100):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h=6,d_model=emb_dim)
    ff = FeedForwardLayer(emb_dim,hid_lay_dim)
    embedding_layer = BertEmbedding()
    positionalLayer = PositionalEncoding(d_model=emb_dim)
    FinalFF = FinalFeedForwardLayer(emb_dim*max_size*2 + 1)
    model = Upgrader(
                RepresentationModule(
                    EncoderModule( EncoderBlock( c(attn), c(ff), emb_dim), N),
                    embedding_layer)
                ,GradingModule(
                    d_model=emb_dim,
                    max_size=max_size,
                    prepLayer=PrepLayer(d_model = emb_dim,
                              max_size = max_size,
                              positionalLayer = positionalLayer),
                    feedForward = FinalFF
                ))
    return model

In [22]:
model = makeModel()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
student_ans = "Sky is red"
question = "What is the colour of the sky?"
reference_ans ="Sky appears blue"
full_marks = 3

In [24]:
marks = model(student_ans,question,reference_ans,full_marks)

torch.Size([1, 1])


In [25]:
marks

tensor([[1.4841]], grad_fn=<MulBackward0>)

In [26]:
def training(model,X,Y):
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
  criterion = nn.L1Loss()
  for i in range(len(X)):
    optimizer.zero_grad()
    Y_hat = model(X[i][0],X[i][1],X[i][2],X[i][3])
    loss = criterion(Y_hat,Y)
    loss.backward()
    optimizer.step()


In [27]:
X = [
     ['Sky is red',"What is the colour of the sky?","Sky appears blue",3],
     ["We can drink water","Can we drink water?","Water is safe to drink",3]
]
Y = torch.range(1,2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  """


In [28]:
training(model,X,Y)

torch.Size([1, 1])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:97: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


torch.Size([1, 1])


In [34]:
Y

tensor([1., 2.])

In [30]:
print(marks.grad_fn)

In [31]:
print(marks)

tensor([[1.4841]], grad_fn=<MulBackward0>)


In [32]:
 p = torch.rand(2)

In [33]:
marks.grad

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  """Entry point for launching an IPython kernel.


In [35]:
import pandas as pd